# **Connect to google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

folder location where your images are

In [ ]:
%cd /content/gdrive/MyDrive/p4p/pasteback_operation/

any folder location where you want to download "shape_predictor_68_face_landmarks" file

In [ ]:
%cd /content/gdrive/MyDrive/p4p/

In [ ]:
!wget   http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 # DOWNLOAD LINK

!bunzip2 /content/gdrive/MyDrive/p4p/shape_predictor_68_face_landmarks.dat.bz2



In [ ]:
datFile =  "/content/gdrive/MyDrive/p4p/shape_predictor_68_face_landmarks.dat"

# **Paste back operation**

In [ ]:
import cv2
import numpy as np
import dlib

# Image paths
img1_path = '/content/gdrive/MyDrive/p4p/pasteback_operation/scarlette.png'         # original image path
img2_path = '/content/gdrive/MyDrive/p4p/pasteback_operation/scarlette_disgust.png'       # cropped image path
output1_path = '/content/gdrive/MyDrive/p4p/pasteback_operation/scarlette_done.jpg'         # output image path

# Load images
img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)

# Initialize dlib's face detector and predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(datFile)

# Get the landmarks
def get_landmarks(image):
    rects = detector(image, 1)
    if len(rects) != 1:
        return None
    return np.matrix([[p.x, p.y] for p in predictor(image, rects[0]).parts()])

# Align the image based on the eyes position
def align_face(image, landmarks):
    left_eye = np.mean(landmarks[36:42], axis=0).tolist()[0]
    right_eye = np.mean(landmarks[42:48], axis=0).tolist()[0]

    dy = right_eye[1] - left_eye[1]
    dx = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(dy, dx))

    h, w = image.shape[:2]
    center = (w // 2, h // 2)

    M = cv2.getRotationMatrix2D(center, angle, 1)
    aligned = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)

    return aligned

# Get landmarks for img1 and aligned face for img2
landmarks1 = get_landmarks(img1)
landmarks2 = get_landmarks(img2)

if landmarks1 is None or landmarks2 is None:
    print("Couldn't detect face landmarks.")
    exit()

aligned_img2 = align_face(img2, landmarks2)

# Extract faces from the images
x1, y1, w1, h1 = cv2.boundingRect(np.array(landmarks1))
x2, y2, w2, h2 = cv2.boundingRect(np.array(landmarks2))

face1 = img1[y1:y1+h1, x1:x1+w1]
face2 = aligned_img2[y2:y2+h2, x2:x2+w2]

# Resize and mask generation
resize_pct = 0.50
face2 = cv2.resize(face2, (int(face2.shape[1] * resize_pct), int(face2.shape[0] * resize_pct)))

def create_soft_mask(face):
    mask = np.ones_like(face) * 255
    h, w, _ = face.shape
    mask[:15, :, :] = 0
    mask[h-15:, :, :] = 0
    mask[:, :15, :] = 0
    mask[:, w-15:, :] = 0
    mask = cv2.GaussianBlur(mask, (51, 51), 0)
    return mask

mask1 = create_soft_mask(face1)
face2_resized = cv2.resize(face2, (w1, h1))
mask2_resized = cv2.resize(create_soft_mask(face2), (w1, h1))
center1 = (x1 + w1 // 2, y1 + h1 // 2)

# Seamless clone
img1_face_swapped = cv2.seamlessClone(face2_resized, img1, mask2_resized, center1, cv2.NORMAL_CLONE)

# Save the results
cv2.imwrite(output1_path, img1_face_swapped)


# **Image to Video**

In [ ]:
import cv2
import numpy as np

# Path to the image and desired output video
image_path = '/content/gdrive/MyDrive/p4p/pasteback_operation/scarlette_done.jpg'       # image path
output_path = '/content/gdrive/MyDrive/p4p/pasteback_operation/scarlette.avi'           # output video path

# Read the image
img = cv2.imread(image_path)
if img is None:
    print(f"Failed to load image at {image_path}")
    exit()

# Extract image dimensions
height, width, layers = img.shape
size = (width, height)

# Define video writer
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

# Write the image frame 60 times for 4 seconds of video at 15 fps
for _ in range(60):
    out.write(img)

out.release()